In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("RetailDemandIngestion") \
    .getOrCreate()


26/01/16 14:34:01 WARN Utils: Your hostname, MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 10.0.0.22 instead (on interface en0)
26/01/16 14:34:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/16 14:34:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import pandas as pd

pdf = pd.read_excel("../data/raw/Online_Retail.xlsx")


df = spark.createDataFrame(pdf)

df.printSchema()
df.show(5)


root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



26/01/16 14:34:35 WARN TaskSetManager: Stage 0 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.
26/01/16 14:34:39 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 0 (TID 0): Attempting to kill Python Worker


+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows



In [3]:
df.createOrReplaceTempView("retail_sales")



In [4]:
spark.sql("""
SELECT
    Country,
    COUNT(DISTINCT InvoiceNo) AS total_orders,
    SUM(Quantity) AS total_quantity,
    ROUND(SUM(Quantity * UnitPrice), 2) AS revenue
FROM retail_sales
GROUP BY Country
ORDER BY revenue DESC
""").show(10)


26/01/16 14:34:40 WARN TaskSetManager: Stage 1 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.


+--------------+------------+--------------+----------+
|       Country|total_orders|total_quantity|   revenue|
+--------------+------------+--------------+----------+
|United Kingdom|       23494|       4263829|8187806.36|
|   Netherlands|         101|        200128| 284661.54|
|          EIRE|         360|        142637| 263276.82|
|       Germany|         603|        117448| 221698.21|
|        France|         461|        110480|  197403.9|
|     Australia|          69|         83653| 137077.27|
|   Switzerland|          74|         30325|  56385.35|
|         Spain|         105|         26824|  54774.58|
|       Belgium|         119|         23152|  40910.96|
|        Sweden|          46|         35637|  36595.91|
+--------------+------------+--------------+----------+
only showing top 10 rows



In [5]:
spark.sql("""
SELECT
    InvoiceNo,
    COUNT(*) AS items_in_invoice
FROM retail_sales
GROUP BY InvoiceNo
ORDER BY items_in_invoice DESC
""").show(5)



26/01/16 14:34:41 WARN TaskSetManager: Stage 7 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.


+---------+----------------+
|InvoiceNo|items_in_invoice|
+---------+----------------+
|   573585|            1114|
|   581219|             749|
|   581492|             731|
|   580729|             721|
|   558475|             705|
+---------+----------------+
only showing top 5 rows



In [6]:
country_sales_df = spark.sql("""
SELECT Country, SUM(Quantity * UnitPrice) AS revenue
FROM retail_sales
GROUP BY Country
""")

country_sales_df.printSchema()


root
 |-- Country: string (nullable = true)
 |-- revenue: double (nullable = true)



In [7]:
spark.sql("SHOW TABLES").show()



+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|         |retail_sales|       true|
+---------+------------+-----------+



In [9]:
df.createOrReplaceTempView("table_name")


In [10]:
spark.sql("SELECT COUNT(*) FROM retail_sales").show()


+--------+
|count(1)|
+--------+
|  541909|
+--------+



26/01/16 14:34:41 WARN TaskSetManager: Stage 13 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.


In [11]:
df.createOrReplaceTempView("retail_sales")


In [12]:
spark.sql("SHOW TABLES").show()


+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|         |retail_sales|       true|
|         |  table_name|       true|
+---------+------------+-----------+



In [13]:
from pyspark.sql.functions import col

df_clean = df.select([
    col(c).alias(c.lower().replace(" ", "_")) for c in df.columns
])

df_clean.printSchema()


root
 |-- invoiceno: string (nullable = true)
 |-- stockcode: string (nullable = true)
 |-- description: string (nullable = true)
 |-- quantity: long (nullable = true)
 |-- invoicedate: timestamp (nullable = true)
 |-- unitprice: double (nullable = true)
 |-- customerid: double (nullable = true)
 |-- country: string (nullable = true)



In [14]:
from pyspark.sql.functions import count, when

df_clean.select([
    count(when(col(c).isNull(), c)).alias(c) for c in df_clean.columns
]).show()


+---------+---------+-----------+--------+-----------+---------+----------+-------+
|invoiceno|stockcode|description|quantity|invoicedate|unitprice|customerid|country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|        0|        0|          0|       0|          0|        0|         0|      0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



26/01/16 14:34:42 WARN TaskSetManager: Stage 16 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.


In [15]:
df_valid = df_clean.filter(
    (col("quantity") > 0) &
    (col("unitprice") > 0) &
    col("invoiceno").isNotNull() &
    col("invoicedate").isNotNull()
)

df_valid.count()


26/01/16 14:34:42 WARN TaskSetManager: Stage 19 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.


530104

In [16]:
from pyspark.sql.functions import when

df_valid = df_valid.withColumn(
    "customerid",
    when(col("customerid").isNull(), -1).otherwise(col("customerid"))
)


In [17]:
df_valid.createOrReplaceTempView("retail_sales_clean")


In [18]:
spark.sql("""
SELECT country, COUNT(*) AS rows_count
FROM retail_sales_clean
GROUP BY country
ORDER BY rows_count DESC
""").show(10)


26/01/16 14:34:42 WARN TaskSetManager: Stage 22 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.


+--------------+----------+
|       country|rows_count|
+--------------+----------+
|United Kingdom|    485123|
|       Germany|      9040|
|        France|      8407|
|          EIRE|      7890|
|         Spain|      2484|
|   Netherlands|      2359|
|       Belgium|      2031|
|   Switzerland|      1966|
|      Portugal|      1501|
|     Australia|      1182|
+--------------+----------+
only showing top 10 rows



In [19]:
from pyspark.sql.functions import round

df_enriched = df_valid.withColumn(
    "revenue",
    round(col("quantity") * col("unitprice"), 2)
)

df_enriched.select("quantity", "unitprice", "revenue").show(5)


+--------+---------+-------+
|quantity|unitprice|revenue|
+--------+---------+-------+
|       6|     2.55|   15.3|
|       6|     3.39|  20.34|
|       8|     2.75|   22.0|
|       6|     3.39|  20.34|
|       6|     3.39|  20.34|
+--------+---------+-------+
only showing top 5 rows



26/01/16 14:34:42 WARN TaskSetManager: Stage 25 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.


In [20]:
df_enriched.write \
    .mode("overwrite") \
    .parquet("../data/processed/retail_sales_clean")


26/01/16 14:34:42 WARN TaskSetManager: Stage 26 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.
26/01/16 14:34:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
26/01/16 14:34:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
26/01/16 14:34:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
26/01/16 14:34:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
26/01/16 14:34:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [21]:
spark.read.parquet("../data/processed/retail_sales_clean").count()


530104

In [22]:
df_enriched.count()


26/01/16 14:34:44 WARN TaskSetManager: Stage 31 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.


530104

In [23]:
df_enriched.createOrReplaceTempView("retail_sales_analytics")


In [24]:
spark.sql("SHOW TABLES").show()


+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|         |        retail_sales|       true|
|         |retail_sales_anal...|       true|
|         |  retail_sales_clean|       true|
|         |          table_name|       true|
+---------+--------------------+-----------+



In [25]:
country_sales = spark.sql("""
SELECT
    country,
    COUNT(DISTINCT invoiceno) AS total_orders,
    SUM(quantity) AS total_quantity,
    ROUND(SUM(revenue), 2) AS total_revenue
FROM retail_sales_analytics
GROUP BY country
ORDER BY total_revenue DESC
""")

country_sales.show(10, truncate=False)



26/01/16 14:34:44 WARN TaskSetManager: Stage 34 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.


+--------------+------------+--------------+-------------+
|country       |total_orders|total_quantity|total_revenue|
+--------------+------------+--------------+-------------+
|United Kingdom|18019       |4662390       |9025222.08   |
|Netherlands   |94          |200361        |285446.34    |
|EIRE          |288         |147173        |283453.96    |
|Germany       |457         |119261        |228867.14    |
|France        |392         |112103        |209715.11    |
|Australia     |57          |83901         |138521.31    |
|Spain         |90          |27940         |61577.11     |
|Switzerland   |54          |30629         |57089.9      |
|Belgium       |98          |23237         |41196.34     |
|Sweden        |36          |36083         |38378.33     |
+--------------+------------+--------------+-------------+
only showing top 10 rows



In [28]:
country_sales.write.mode("overwrite") \
    .parquet("../data/processed/country_sales")

daily_demand.write.mode("overwrite") \
    .parquet("../data/processed/daily_demand")

product_demand.write.mode("overwrite") \
    .parquet("../data/processed/product_demand")


26/01/16 14:34:45 WARN TaskSetManager: Stage 46 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.
26/01/16 14:34:45 WARN TaskSetManager: Stage 59 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.
26/01/16 14:34:46 WARN TaskSetManager: Stage 67 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.


In [29]:
spark.read.parquet("../data/processed/daily_demand").show(5)


+----------+--------------+-------------+
|sales_date|daily_quantity|daily_revenue|
+----------+--------------+-------------+
|2010-12-01|         26919|     58960.79|
|2010-12-02|         31329|     47748.38|
|2010-12-03|         16199|     46943.71|
|2010-12-05|         16450|     31774.95|
|2010-12-06|         21795|     54830.46|
+----------+--------------+-------------+
only showing top 5 rows



In [30]:
from pyspark.sql.functions import dayofweek, weekofyear, month, year

daily_features = daily_demand \
    .withColumn("day_of_week", dayofweek("sales_date")) \
    .withColumn("week_of_year", weekofyear("sales_date")) \
    .withColumn("month", month("sales_date")) \
    .withColumn("year", year("sales_date"))

daily_features.show(5)


+----------+--------------+-------------+-----------+------------+-----+----+
|sales_date|daily_quantity|daily_revenue|day_of_week|week_of_year|month|year|
+----------+--------------+-------------+-----------+------------+-----+----+
|2010-12-01|         26919|     58960.79|          4|          48|   12|2010|
|2010-12-02|         31329|     47748.38|          5|          48|   12|2010|
|2010-12-03|         16199|     46943.71|          6|          48|   12|2010|
|2010-12-05|         16450|     31774.95|          1|          48|   12|2010|
|2010-12-06|         21795|     54830.46|          2|          49|   12|2010|
+----------+--------------+-------------+-----------+------------+-----+----+
only showing top 5 rows



26/01/16 14:34:46 WARN TaskSetManager: Stage 77 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.


In [31]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag

window_spec = Window.orderBy("sales_date")

daily_features = daily_features \
    .withColumn("lag_1_day", lag("daily_quantity", 1).over(window_spec)) \
    .withColumn("lag_7_day", lag("daily_quantity", 7).over(window_spec))


In [32]:
daily_features.select(
    "sales_date", "daily_quantity", "lag_1_day", "lag_7_day"
).show(10)


26/01/16 14:34:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:46 WARN TaskSetManager: Stage 80 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.
26/01/16 14:34:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+--------------+---------+---------+
|sales_date|daily_quantity|lag_1_day|lag_7_day|
+----------+--------------+---------+---------+
|2010-12-01|         26919|     NULL|     NULL|
|2010-12-02|         31329|    26919|     NULL|
|2010-12-03|         16199|    31329|     NULL|
|2010-12-05|         16450|    16199|     NULL|
|2010-12-06|         21795|    16450|     NULL|
|2010-12-07|         25220|    21795|     NULL|
|2010-12-08|         23117|    25220|     NULL|
|2010-12-09|         19930|    23117|    26919|
|2010-12-10|         21097|    19930|    31329|
|2010-12-12|         10603|    21097|    16199|
+----------+--------------+---------+---------+
only showing top 10 rows



26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [33]:
from pyspark.sql.functions import avg

rolling_window = window_spec.rowsBetween(-7, -1)

daily_features = daily_features.withColumn(
    "rolling_7_day_avg",
    avg("daily_quantity").over(rolling_window)
)

daily_features.select(
    "sales_date", "daily_quantity", "rolling_7_day_avg"
).show(10)


26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:47 WARN TaskSetManager: Stage 92 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.
26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:47 WARN WindowExec

+----------+--------------+------------------+
|sales_date|daily_quantity| rolling_7_day_avg|
+----------+--------------+------------------+
|2010-12-01|         26919|              NULL|
|2010-12-02|         31329|           26919.0|
|2010-12-03|         16199|           29124.0|
|2010-12-05|         16450|24815.666666666668|
|2010-12-06|         21795|          22724.25|
|2010-12-07|         25220|           22538.4|
|2010-12-08|         23117|22985.333333333332|
|2010-12-09|         19930| 23004.14285714286|
|2010-12-10|         21097|22005.714285714286|
|2010-12-12|         10603|           20544.0|
+----------+--------------+------------------+
only showing top 10 rows



26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [34]:
daily_features_clean = daily_features.dropna()
daily_features_clean.count()


26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:47 WARN TaskSetManager: Stage 104 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.
26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:47 WARN WindowExe

298

In [35]:
daily_features_clean.write \
    .mode("overwrite") \
    .parquet("../data/processed/daily_features")


26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:47 WARN TaskSetManager: Stage 116 contains a task of very large size (3431 KiB). The maximum recommended task size is 1000 KiB.
26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/16 14:34:47 WARN WindowExe

In [36]:
spark.read.parquet("../data/processed/daily_features").show(5)


+----------+--------------+-------------+-----------+------------+-----+----+---------+---------+------------------+
|sales_date|daily_quantity|daily_revenue|day_of_week|week_of_year|month|year|lag_1_day|lag_7_day| rolling_7_day_avg|
+----------+--------------+-------------+-----------+------------+-----+----+---------+---------+------------------+
|2010-12-09|         19930|     53586.18|          5|          49|   12|2010|    23117|    26919| 23004.14285714286|
|2010-12-10|         21097|     59182.92|          6|          49|   12|2010|    19930|    31329|22005.714285714286|
|2010-12-12|         10603|     17329.07|          1|          49|   12|2010|    21097|    16199|           20544.0|
|2010-12-13|         17727|     38006.71|          2|          50|   12|2010|    10603|    16450|19744.571428571428|
|2010-12-14|         20284|     45254.73|          3|          50|   12|2010|    17727|    21795|           19927.0|
+----------+--------------+-------------+-----------+-----------